## Deploy to a real-time endpoint

### Deploy with Python SDK

An `Estimator` could be deployed directly after training, with an `Estimator.deploy()` but here we showcase the more extensive process of creating a model from s3 artifacts, that could be used to deploy a model that was trained in a different session or even out of SageMaker.

In [5]:
import boto3
from sagemaker import get_execution_role
import sagemaker

sm_boto3 = boto3.client('sagemaker')

sess = sagemaker.Session()

FRAMEWORK_VERSION = '0.23-1'

In [6]:
training_job_name = "RF-tuner-210303-1606-008-927dd354"
artifact = sm_boto3.describe_training_job(
    TrainingJobName=training_job_name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)

Model artifact persisted at s3://sagemaker-us-west-1-365792799466/RF-tuner-210303-1606-008-927dd354/output/model.tar.gz


In [7]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point='script.py',
    framework_version=FRAMEWORK_VERSION)

In [9]:
predictor = model.deploy(
    instance_type='ml.t2.medium',
    initial_instance_count=1)

-----------------!

### Invoke with the Python SDK

In [11]:
import datetime
import time
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston


sm_boto3 = boto3.client('sagemaker')

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print('Using bucket ' + bucket)

# we use the Boston housing dataset 
data = load_boston()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

testX = pd.DataFrame(X_test, columns=data.feature_names)
testX['target'] = y_test


Using bucket sagemaker-us-west-1-365792799466


In [12]:
# the SKLearnPredictor does the serialization from pandas for us
print(predictor.predict(testX[data.feature_names]))

[23.16920942 32.15828685 16.50942366 24.07429417 17.16555197 21.1998417
 19.29227288 15.7307995  20.96383677 21.13370338 19.39466318 19.78174356
  8.39423967 21.55040396 20.11220673 27.13923488 18.18162667  8.81443018
 44.32863202 15.20816613 24.15842348 23.9706319  15.02521982 23.6974462
 14.61674858 15.25011126 21.24043037 14.6236045  19.35731711 20.78159795
 19.82000884 23.56228987 28.14442107 20.8264674  14.53748271 15.77383415
 35.84300106 19.80514047 20.7594846  24.17252452 19.21432294 29.00699257
 44.36556364 19.62437325 22.67122216 14.12152853 15.64635549 24.21466759
 18.42225787 28.7422653  20.95245021 34.31429461 16.81095781 26.43608613
 45.50720732 21.25199084 15.61377742 33.11559341 22.14910967 20.34006623
 25.25066041 34.67889479 31.77414764 19.28390668 27.61154228 16.51844048
 13.16436097 23.25194464 29.17577577 15.47115818 20.52625905 26.81027485
  9.88534815 22.9849072  21.73566246  8.57009734 19.70447606 44.36963202
 11.58589946 14.15743585 21.41942367 11.54895008 20.3

### Alternative: invoke with `boto3`

In [13]:
runtime = boto3.client('sagemaker-runtime')

#### Option 1: `csv` serialization

In [14]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode('utf-8'),
    ContentType='text/csv')

print(response['Body'].read())

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


b'[23.16920941897799, 32.158286848419905, 16.509423658286156, 24.07429416852908, 17.165551967129016, 21.199841703184163, 19.292272883105994, 15.730799496444384, 20.963836774530076, 21.13370337846278, 19.394663184738377, 19.7817435561055, 8.394239670958562, 21.550403963043326, 20.112206732771003, 27.13923488286309, 18.181626666384872, 8.814430177980775, 44.32863201912467, 15.208166134656535, 24.158423475814505, 23.970631903803113, 15.025219817171132, 23.697446200123412, 14.61674858499344, 15.250111257069381, 21.240430369226573, 14.623604503421419, 19.35731711398337, 20.78159795115016, 19.820008837384258, 23.56228986683681, 28.144421068504077, 20.826467396372124, 14.537482706638048, 15.773834149360276, 35.84300106389139, 19.805140470236676, 20.759484600040484, 24.172524521475246, 19.214322939048508, 29.006992568989336, 44.36556364305629, 19.624373253771516, 22.67122216179932, 14.121528530031206, 15.646355492851661, 24.21466759076387, 18.42225787432943, 28.742265302280273, 20.952450207386

#### Option 2: `npy` serialization

In [15]:
# npy serialization
from io import BytesIO


#Serialise numpy ndarray as bytes
buffer = BytesIO()
# Assuming testX is a data frame
np.save(buffer, testX[data.feature_names].values)

response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint,
    Body=buffer.getvalue(),
    ContentType='application/x-npy')

print(response['Body'].read())

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


b'[23.16920941897799, 32.158286848419905, 16.509423658286156, 24.07429416852908, 17.165551967129016, 21.199841703184163, 19.292272883105994, 15.730799496444384, 20.963836774530076, 21.13370337846278, 19.394663184738377, 19.7817435561055, 8.394239670958562, 21.550403963043326, 20.112206732771003, 27.13923488286309, 18.181626666384872, 8.814430177980775, 44.32863201912467, 15.208166134656535, 24.158423475814505, 23.970631903803113, 15.025219817171132, 23.697446200123412, 14.61674858499344, 15.250111257069381, 21.240430369226573, 14.623604503421419, 19.35731711398337, 20.78159795115016, 19.820008837384258, 23.56228986683681, 28.144421068504077, 20.826467396372124, 14.537482706638048, 15.773834149360276, 35.84300106389139, 19.805140470236676, 20.759484600040484, 24.172524521475246, 19.214322939048508, 29.006992568989336, 44.36556364305629, 19.624373253771516, 22.67122216179932, 14.121528530031206, 15.646355492851661, 24.21466759076387, 18.42225787432943, 28.742265302280273, 20.952450207386

## Don't forget to delete the endpoint !

In [16]:
sm_boto3.delete_endpoint(EndpointName=predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'ResponseMetadata': {'RequestId': '77c8cfea-e973-41d6-8089-ca3082c59ec7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '77c8cfea-e973-41d6-8089-ca3082c59ec7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 03 Mar 2021 18:27:15 GMT'},
  'RetryAttempts': 0}}